In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib 
# from rdkit import Chem
import os

from torch_geometric.loader import DataLoader
import torch
from utils_data import TestbedDataset
from utils_preproc import *

In [34]:
branch = 'decode_attn'
path = 'root_folder/root_' + branch

In [19]:
cell_dict, cell_feature, genes = save_gene_expr_matrix_X(filter_by_l1000=True)
# cell_dict

number of preserved landmark genes: 956


In [20]:
genes

Index(['ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1', 'ACAA1',
       'ACAT2', 'ACBD3',
       ...
       'ZMIZ1', 'ZMYM2', 'ZNF131', 'ZNF274', 'ZNF318', 'ZNF395', 'ZNF451',
       'ZNF586', 'ZNF589', 'ZW10'],
      dtype='object', length=956)

In [21]:
cell_feature.shape

(700, 956)

In [22]:
genes = genes.values
gene_list = np.array([g.split(' (')[0] for g in genes])

In [23]:
gene_list

array(['ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1',
       'ACAA1', 'ACAT2', 'ACBD3', 'ACD', 'ACLY', 'ACOT9', 'ADAM10',
       'ADAT1', 'ADGRE5', 'ADGRG1', 'ADH5', 'ADI1', 'ADO', 'ADRB2', 'AGL',
       'AKAP8', 'AKAP8L', 'AKR7A2', 'AKT1', 'ALAS1', 'ALDH7A1', 'ALDOA',
       'ALDOC', 'AMDHD2', 'ANKRD10', 'ANO10', 'ANXA7', 'APBB2', 'APOE',
       'APP', 'APPBP2', 'ARFIP2', 'ARHGAP1', 'ARHGEF12', 'ARHGEF2',
       'ARID4B', 'ARID5B', 'ARL4C', 'ARNT2', 'ARPP19', 'ASAH1', 'ASCC3',
       'ATF1', 'ATF5', 'ATF6', 'ATG3', 'ATMIN', 'ATP11B', 'ATP1B1',
       'ATP2C1', 'ATP6V0B', 'ATP6V1D', 'AURKA', 'AURKB', 'AXIN1',
       'B4GAT1', 'BACE2', 'BAD', 'BAG3', 'BAMBI', 'BAX', 'BCL2', 'BCL7B',
       'BDH1', 'BECN1', 'BHLHE40', 'BID', 'BIRC2', 'BIRC5', 'BLCAP',
       'BLMH', 'BLVRA', 'BMP4', 'BNIP3', 'BNIP3L', 'BPHL', 'BRCA1', 'BTK',
       'BUB1B', 'BZW2', 'C2CD2', 'C2CD2L', 'C2CD5', 'C5', 'CAB39',
       'CALM3', 'CALU', 'CAMSAP2', 'CANT1', 'CAPN1', 'CARMIL1', 'CASC3',
       'CA

In [42]:
cell_feature[0].shape

(956,)

In [43]:
ge_df = pd.DataFrame(cell_feature, index=list(cell_dict.keys()), columns=gene_list)
ge_df

,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,ACBD3,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
687455,1.735522,6.638943,6.184479,5.447579,3.747387,1.786596,4.311794,6.907251,6.435795,4.448240,...,4.093391,4.211791,4.483493,3.179511,2.353323,4.832384,3.665620,3.744161,2.032101,4.127633
1659929,4.104337,3.371559,5.019257,5.147307,2.608809,3.929791,4.783457,6.542413,6.352617,4.704319,...,3.669027,4.737687,3.929791,2.759156,2.935460,4.023255,3.378512,2.367371,2.985500,4.195348
684681,3.987321,3.353323,5.537296,4.942984,3.489286,1.356144,3.240314,5.602884,4.721373,4.899659,...,1.244887,3.364572,4.526069,4.039138,1.910733,3.732269,3.190615,2.601697,1.879706,2.967169
906836,1.803227,4.096768,6.151575,5.397118,3.839960,0.985500,4.540399,4.718636,4.915999,5.965784,...,4.031219,4.954662,5.178316,3.436961,3.418190,5.610287,4.257765,3.003602,3.647315,3.308885
906791,1.384050,7.301039,5.852498,6.490570,3.866908,1.799087,4.605850,5.180307,7.131343,4.993674,...,1.367371,4.669594,5.338781,3.374344,3.405992,3.456806,3.599318,3.244887,1.918386,4.653060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240218,4.786596,3.498251,4.865424,5.183089,5.093391,1.485427,3.165108,4.780310,5.246788,5.173127,...,3.282440,5.336997,4.016140,3.838952,3.046142,3.082362,3.386811,0.895303,2.266037,3.681449
908441,3.697107,5.043519,5.431623,6.080444,3.887525,1.650765,4.084064,4.345538,7.281049,4.334139,...,5.204767,5.468909,5.464668,4.167519,2.972693,3.078951,4.123501,3.204767,5.377124,4.118526
906798,3.341986,4.764474,6.023255,5.806324,2.889474,3.124328,4.421560,5.694602,6.676098,4.257011,...,2.565597,4.570463,5.956289,4.038261,4.116032,2.327687,4.290572,2.543496,4.090853,5.005400
905974,3.745237,4.454176,6.019035,5.028127,3.379898,1.739848,6.607182,4.969933,6.007196,4.114367,...,3.593354,4.975905,4.455492,4.033863,2.887525,3.890447,3.831877,2.978196,3.243364,4.361768


In [45]:
# tmp = []
iqr = ge_df.quantile(0.75) - ge_df.quantile(0.25)
iqr

ABCB6     1.733997
ABCC5     1.098072
ABCF1     0.792157
ABCF3     0.639471
ABHD4     1.502482
            ...   
ZNF395    1.146133
ZNF451    0.750704
ZNF586    0.908604
ZNF589    1.334128
ZW10      0.782605
Length: 956, dtype: float64

In [46]:
lim = np.abs((ge_df - ge_df.median()) / iqr) < 2.22
filtered_ge_df = ge_df.where(lim, np.nan)
filtered_ge_df

,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,ACBD3,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
687455,1.735522,6.638943,6.184479,5.447579,3.747387,1.786596,4.311794,6.907251,6.435795,4.448240,...,4.093391,4.211791,4.483493,3.179511,2.353323,4.832384,3.665620,3.744161,2.032101,4.127633
1659929,4.104337,3.371559,5.019257,5.147307,2.608809,3.929791,4.783457,6.542413,6.352617,4.704319,...,3.669027,4.737687,3.929791,2.759156,2.935460,4.023255,3.378512,2.367371,2.985500,4.195348
684681,3.987321,3.353323,5.537296,4.942984,3.489286,1.356144,3.240314,5.602884,4.721373,4.899659,...,1.244887,3.364572,4.526069,4.039138,1.910733,3.732269,3.190615,2.601697,1.879706,2.967169
906836,1.803227,4.096768,6.151575,5.397118,3.839960,0.985500,4.540399,4.718636,4.915999,5.965784,...,4.031219,4.954662,5.178316,3.436961,3.418190,5.610287,4.257765,3.003602,3.647315,3.308885
906791,1.384050,NaN,5.852498,6.490570,3.866908,1.799087,4.605850,5.180307,7.131343,4.993674,...,1.367371,4.669594,5.338781,3.374344,3.405992,3.456806,3.599318,3.244887,1.918386,4.653060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240218,4.786596,3.498251,4.865424,5.183089,5.093391,1.485427,3.165108,4.780310,5.246788,5.173127,...,3.282440,5.336997,4.016140,3.838952,3.046142,3.082362,3.386811,0.895303,2.266037,3.681449
908441,3.697107,5.043519,5.431623,6.080444,3.887525,1.650765,4.084064,4.345538,7.281049,4.334139,...,5.204767,5.468909,5.464668,4.167519,2.972693,3.078951,4.123501,3.204767,5.377124,4.118526
906798,3.341986,4.764474,6.023255,5.806324,2.889474,3.124328,4.421560,5.694602,6.676098,4.257011,...,2.565597,4.570463,5.956289,4.038261,4.116032,2.327687,4.290572,2.543496,4.090853,5.005400
905974,3.745237,4.454176,6.019035,5.028127,3.379898,1.739848,6.607182,4.969933,6.007196,4.114367,...,3.593354,4.975905,4.455492,4.033863,2.887525,3.890447,3.831877,2.978196,3.243364,4.361768


In [47]:
filtered_ge_df.mean(axis=0, skipna=True)

ABCB6     4.187672
ABCC5     4.389618
ABCF1     5.761267
ABCF3     5.469246
ABHD4     3.684390
            ...   
ZNF395    4.085327
ZNF451    3.890610
ZNF586    2.798240
ZNF589    2.913565
ZW10      3.969414
Length: 956, dtype: float64

In [48]:
gene_stats = ge_df.T
gene_stats['min'] = ge_df.min()
gene_stats['Q1'] = ge_df.quantile(0.25)
gene_stats['median'] = ge_df.quantile(0.5)
gene_stats['Q3'] = ge_df.quantile(0.75)
gene_stats['max'] = ge_df.max()
gene_stats['ori_mean'] = ge_df.mean(axis=0)
gene_stats['filtered_mean'] = filtered_ge_df.mean(axis=0, skipna=True)
gene_stats = gene_stats.iloc[:, -7:]
gene_stats

,min,Q1,median,Q3,max,ori_mean,filtered_mean
ABCB6,0.189034,3.368766,4.346957,5.102763,7.858913,4.181960,4.187672
ABCC5,0.713696,3.822220,4.353323,4.920292,8.295723,4.401509,4.389618
ABCF1,2.204767,5.370687,5.782011,6.162844,7.631614,5.732454,5.761267
ABCF3,2.939227,5.142413,5.482364,5.781884,7.258236,5.463421,5.469246
ABHD4,0.000000,2.903508,3.677507,4.405990,7.053872,3.653924,3.684390
...,...,...,...,...,...,...,...
ZNF395,0.084064,3.521049,4.076816,4.667182,7.458776,4.067620,4.085327
ZNF451,1.627607,3.523555,3.881175,4.274259,6.390083,3.884926,3.890610
ZNF586,0.000000,2.336283,2.790772,3.244887,4.789208,2.755023,2.798240
ZNF589,0.815575,2.235727,2.911691,3.569855,5.377124,2.913565,2.913565


In [49]:
gene_stats.to_csv('data/gene_statistics.csv')

In [13]:
gene_stats['filtered_mean'].values

array([5.56079396, 4.77718817, 4.73901584, ..., 0.08008788, 0.10234567,
       0.08751044])

In [14]:
drug_dict, drug_smile, smile_graph = load_drug_smile_X()
# drug_dict

### Debug missing genes from CCLE_expression to gene_statistics

In [13]:
ccle_expr_df = pd.read_csv('data/CCLE/CCLE_expression.csv', header=0, index_col=0)

In [12]:
ccle_expr_df['EPHA3 (2042)']

ACH-001113    0.056584
ACH-001289    1.090853
ACH-001339    2.863938
ACH-001538    0.831877
ACH-000242    0.084064
                ...   
ACH-000285    0.084064
ACH-002669    0.000000
ACH-001858    0.014355
ACH-001997    0.000000
ACH-000052    3.238787
Name: EPHA3 (2042), Length: 1406, dtype: float64

In [22]:
ccle_genes = ccle_expr_df.columns.values
np.where(ccle_genes=='EPHA3 (2042)')

(array([601]),)

In [35]:
ccle_genes_rename = np.array([g.split(' (')[0] for g in ccle_genes])
np.where(ccle_genes_rename=='EPHA3')

(array([601]),)

In [14]:
ge_df

,KRT19,SPARC,C19orf33,VIM,UCHL1,TGFBI,KRT7,FN1,KRT8,FXYD3,...,RPEL1,ECT2L,HNRNPCL1,KIF4B,FAM186A,SPDYE3,PRH2,CAGE1,PPP1R2B,AC090517.4
687455,10.970609,0.739848,8.044886,1.748461,1.384050,3.190615,11.812258,3.427606,10.431393,9.470191,...,0.189034,0.097611,0.163499,0.097611,0.014355,0.150560,0.070389,0.084064,0.111031,0.163499
1659929,9.360715,2.632268,7.482203,3.140779,1.150560,5.874059,2.060047,3.307429,11.374116,8.630267,...,0.124328,0.070389,0.056584,0.042644,0.014355,0.070389,0.028569,0.056584,0.056584,0.056584
684681,4.431623,0.823749,6.802839,1.475085,5.741197,6.598574,9.222867,4.015248,9.051019,7.455163,...,0.056584,0.042644,0.014355,0.070389,0.070389,0.056584,0.000000,0.070389,0.042644,0.070389
906836,2.000000,1.124328,0.956057,9.580485,1.304511,2.049631,1.432959,2.879706,3.798051,0.704872,...,0.189034,0.028569,0.613532,0.298658,0.137504,0.201634,0.097611,0.163499,0.176323,0.056584
906791,10.287712,1.327687,9.001239,1.464668,0.411426,1.084064,4.766595,8.755455,11.672779,3.636915,...,0.124328,0.137504,0.214125,0.042644,0.214125,0.070389,0.056584,0.042644,0.163499,0.070389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240218,9.385560,2.090853,8.680887,4.865919,4.989139,8.011898,10.982352,8.619340,10.996403,7.676803,...,0.189034,0.137504,0.056584,0.028569,0.084064,0.150560,0.286881,0.028569,0.084064,0.042644
908441,1.427606,5.685099,0.000000,9.353566,7.783522,2.292782,0.176323,5.436961,5.659354,0.014355,...,0.014355,0.042644,0.250962,0.070389,0.000000,0.042644,0.028569,0.201634,0.084064,0.000000
906798,3.737687,1.042644,5.580447,9.206355,9.000056,2.350497,9.889413,6.465322,10.390491,0.782409,...,0.084064,0.042644,0.250962,0.163499,0.056584,0.084064,0.097611,0.163499,0.124328,0.042644
905974,1.007196,7.974529,0.422233,11.202675,0.333424,6.248876,1.521051,7.721373,6.010108,0.495695,...,0.084064,0.042644,0.084064,0.084064,0.056584,0.042644,0.111031,0.056584,0.028569,0.084064


In [44]:
np.where(gene_list=='EPHA3')

(array([271]),)

In [34]:
gene_stats.loc['EPHA3']

KeyError: 'EPHA3'

# Explore salient genes of most sensitive drugs
## Find most sensitive drugs

In [6]:
ic50 = pd.read_csv('data/GDSC/PANCANCER_IC.csv', usecols=['Drug name', 'IC50'])
ic50_avg = ic50.groupby(['Drug name']).mean()
ic50_avg

,IC50
Drug name,
(5Z)-7-Oxozeaenol,0.817922
5-Fluorouracil,3.565578
681640,2.117400
A-443654,-0.739381
A-770041,2.083244
...,...
ZG-10,1.495344
ZM447439,2.216697
ZSTK474,1.016306


In [7]:
ic50_std = ic50.groupby(['Drug name']).std()
ic50_std

,IC50
Drug name,
(5Z)-7-Oxozeaenol,1.417219
5-Fluorouracil,1.767881
681640,1.101806
A-443654,1.379241
A-770041,1.791526
...,...
ZG-10,1.203072
ZM447439,1.264389
ZSTK474,1.707768


In [8]:
sensitivity = ic50_avg.merge(ic50_std, on=['Drug name'])
sensitivity = sensitivity.sort_values(by=['IC50_x'])
sensitivity = sensitivity.rename(columns={'IC50_x':'IC50_avg', 'IC50_y':'IC50_std'})
sensitivity

,IC50_avg,IC50_std
Drug name,,
Bortezomib,-5.647788,1.565107
Docetaxel,-4.901555,1.516071
Epothilone B,-4.861551,1.879182
SN-38,-4.240695,1.671931
Camptothecin,-4.060004,1.790315
...,...,...
Temozolomide,5.535896,0.805496
Cetuximab,5.981718,1.007772
DMOG,6.336191,1.443846


In [9]:
# sensitivity.to_csv('data/drug_sensitivity.csv')

In [10]:
sorted_ic50_avg = ic50_avg.sort_values(by=['IC50'])
sorted_ic50_avg.loc[sorted_ic50_avg['IC50']<=0]

,IC50
Drug name,
Bortezomib,-5.647788
Docetaxel,-4.901555
Epothilone B,-4.861551
SN-38,-4.240695
Camptothecin,-4.060004
Vinblastine,-4.049858
Thapsigargin,-3.830077
Vinorelbine,-3.797435
Sepantronium bromide,-3.685045


In [11]:
# selected_drugs = sorted_ic50_avg.loc[sorted_ic50_avg['IC50']<=0].index
# pd.DataFrame(selected_drugs.values).to_csv('data/sensitive_drugs.csv', index=None, header=None)

# pd.DataFrame(sorted_ic50_avg.index.values).to_csv('data/all_drugs.csv', index=None, header=None)

In [12]:
sorted_ic50_std = ic50_std.loc[sorted_ic50_avg.index]
# sorted_ic50_std.head(selected_drugs.shape[0])

In [13]:
# n_d = selected_drugs.shape[0]
# d_list = sorted_ic50_avg.index[:n_d].values

d_list = sorted_ic50_avg.index.values
d_list

array(['Bortezomib', 'Docetaxel', 'Epothilone B', 'SN-38', 'Camptothecin',
       'Vinblastine', 'Thapsigargin', 'Vinorelbine ',
       'Sepantronium bromide', 'Paclitaxel', 'Elesclomol', 'Omipalisib',
       'Bryostatin 1', 'Luminespib', 'Dacinostat', 'Daporinad',
       'GW843682X', 'Dactolisib', 'Rapamycin', 'Gemcitabine', 'CGP-60474',
       'Temsirolimus', 'BI-2536', 'Ispinesib Mesylate', 'Doxorubicin',
       'JW-7-52-1', 'THZ-2-102-1', 'QL-VIII-58', 'PD0325901',
       'Trametinib', 'Obatoclax Mesylate', 'rTRAIL', 'A-443654',
       'Mitomycin-C', 'Lestauritinib', 'SNX-2112', 'Tanespimycin',
       'TW 37', 'Methotrexate', 'AZD7762', 'CUDC-101', 'AR-42', 'JNK-9L',
       'MG-132', 'IOX2', 'AZD8055', 'Midostaurin', 'Shikonin', 'WZ3105',
       'Cytarabine', 'Pevonedistat', 'WIKI4', 'Belinostat', 'Tivozanib',
       'Dasatinib', 'Ponatinib', 'BMS-754807', 'Entinostat', 'NVP-TAE684',
       'Z-LLNle-CHO', '(5Z)-7-Oxozeaenol', 'CAY10603', 'Tozasertib',
       'Pictilisib', 'Vorinost

## load saliency score and ranks
select from drug list (sensitive ones or all)

In [3]:
from utils_decoding import *
# matplotlib.rc('xtick', labelsize=40) 
# matplotlib.rc('ytick', labelsize=40) 

In [41]:
model_name = 'GAT'
ss_path = path + '/Saliency/IG/CellLine/' + model_name + '/iqr_mean_baseline'

drug_dict, sal_dict =  make_gene_ss_dict(ss_path, type='drug')
norm_sal_dict = normalize_ss(sal_dict)
rank_dict = rank_ss(norm_sal_dict)

In [42]:
print(len(rank_dict['Lenalidomide']), len(norm_sal_dict), len(drug_dict))

956 223 223


In [43]:
def select_dict(ori_dict, key_list):
    return {k:v for k, v in ori_dict.items() if k in key_list}

In [44]:
selected_sal_dict = select_dict(norm_sal_dict, d_list)
selected_drug_dict = select_dict(drug_dict, d_list)
selected_rank_dict = select_dict(rank_dict, d_list)

## Drug gene interaction

### first let's find all drug gene interaction pairs in our dataset
among all our drugs and all the genes
- 2400 pairs in total
- out of 3202 pairs from dgi database)

In [17]:
# interac = pd.read_csv('data/sensitive_drug_gene_interaction.tsv', sep='\t', usecols=['search_term', 'gene'])
interac = pd.read_csv('data/all_drug_gene_interaction.tsv', sep='\t', usecols=['search_term', 'gene'])

interac = interac.set_index('search_term')
interac

,gene
search_term,
BORTEZOMIB,PSMC2
BORTEZOMIB,GATA2
BORTEZOMIB,PSMD4P1
BORTEZOMIB,PSMD11
BORTEZOMIB,PSMB1
...,...
CETUXIMAB,KDR
CETUXIMAB,IDH1
PHENFORMIN,STK11


In [45]:
l_interac_all = []
for k, v in rank_dict.items():
    ranked_genes = gene_list[v]
    sal_score = norm_sal_dict[k].reshape(-1)
    ranked_ss = sal_score[v]
    if k.upper() not in interac.index:
        continue
    drug_gene = interac.loc[k.upper()].values
    
#     for g in ranked_genes:
    for i in range(len(ranked_genes)):
        g = ranked_genes[i]
        ss = ranked_ss[i]
        if g in drug_gene:
#             print(k, g)
            l_interac_all.append([k, g, i, ss])
            
interac_all_df = pd.DataFrame(l_interac_all, columns=['drug', 'gene', 'rank', 'sal_score']).set_index('drug')
interac_all_df

,gene,rank,sal_score
drug,,,
Tivozanib,ERBB2,69,0.253954
PLX-4720,NRAS,764,0.039121
CUDC-101,ERBB2,35,0.321997
CUDC-101,EGFR,41,0.295495
CUDC-101,HDAC2,664,0.052144
...,...,...,...
Dasatinib,CRKL,903,0.016115
Dasatinib,ABL1,938,0.004436
Parthenolide,NFKB2,291,0.059762


In [46]:
# interac_all_df.to_csv('root_folder/root_069/Saliency/IG/CellLine/GAT/iqr_mean_baseline_existing_dgi.csv')
interac_all_df.to_csv(path + '/Saliency/IG/CellLine/' + model_name + '/iqr_mean_baseline_existing_dgi.csv')

In [29]:
# top_n = 100
# for k, v in selected_rank_dict.items():
#     ranked_genes = gene_list[v][:top_n]
#     print(k, ranked_genes)

In [21]:
list(set(interac.index.values))

['ALECTINIB',
 'PELITINIB',
 'TAMOXIFEN',
 'AMUVATINIB',
 'PF-562271',
 'TEMOZOLOMIDE',
 'CAMPTOTHECIN',
 'TOZASERTIB',
 'PI-103',
 'BICALUTAMIDE',
 'AVAGACESTAT',
 'IMATINIB',
 'Y-39983',
 'FEDRATINIB',
 'CYTARABINE',
 'S-TRITYL-L-CYSTEINE',
 'BMS-754807',
 'LINIFANIB',
 'OMIPALISIB',
 'LUMINESPIB',
 'ENTINOSTAT',
 'DACINOSTAT',
 'BMS-345541',
 'SELUMETINIB',
 'SERDEMETAN',
 'ZIBOTENTAN',
 'BI-2536',
 'DOCETAXEL',
 'ERLOTINIB',
 'BORTEZOMIB',
 'QUIZARTINIB',
 'TUBASTATIN A',
 'RUXOLITINIB',
 'CUDC-101',
 'TIPIFARNIB',
 'SARACATINIB',
 'TRAMETINIB',
 'MOTESANIB',
 'SB590885',
 'PALBOCICLIB',
 'CISPLATIN',
 'CABOZANTINIB',
 'GSK690693',
 'MG-132',
 'LAPATINIB',
 'SN-38',
 'EMBELIN',
 'PAZOPANIB',
 'DASATINIB',
 'TANESPIMYCIN',
 '5-FLUOROURACIL',
 'BOSUTINIB',
 'GEMCITABINE',
 'MK-2206',
 'BLEOMYCIN',
 'ENZASTAURIN',
 'VELIPARIB',
 'DORAMAPIMOD',
 'T0901317',
 'THAPSIGARGIN',
 'OBATOCLAX MESYLATE',
 'NAVITOCLAX',
 'VORINOSTAT',
 'AZD6482',
 'CGP-60474',
 'TRETINOIN',
 'IDELALISIB',
 'BRY

In [25]:
top_n = 13143
for k, v in selected_rank_dict.items():
    ranked_genes = gene_list[v][:top_n]
    if k.upper() not in interac.index:
        continue
    drug_gene = interac.loc[k.upper()].values
    
    for g in ranked_genes:
        if g in drug_gene:
            print(k, g)
#     print(k, ranked_genes)

Camptothecin GSTP1
Camptothecin TGM2
Camptothecin COL18A1
Camptothecin THBS1
SN-38 BCHE
SN-38 CES1
SN-38 ABCC2
Axitinib PDGFRB
Axitinib PDGFRA
Pazopanib HLA-B
Pazopanib MET
Pazopanib FGFR1
Pazopanib FYN
Pazopanib FGFR2
Pazopanib PDGFRB
Pazopanib PDGFRA
NSC-87877 ADM
NSC-87877 PTPN6
Bicalutamide FST
Etoposide GSTP1
Etoposide GDF15
Etoposide FGFR1
Etoposide EGFR
Etoposide IGF2
PF-562271 MET
PF-562271 EPHA2
PF-562271 FGFR1
PF-562271 AXL
PF-562271 FYN
PF-562271 MST1R
PF-562271 PDGFRB
PF-562271 BTK
Selumetinib MET
Selumetinib FGFR1
Selumetinib EGFR
Selumetinib ERBB3
Selumetinib CDH1
Selumetinib FGFR2
Dactolisib EGFR
Dactolisib MET
AZD8055 FGFR1
Cytarabine GSTP1
Cytarabine BMP7
Cytarabine GATA3
Cytarabine CDA
Pictilisib FGFR2
Doramapimod EGFR
Doramapimod PDGFRB
Amuvatinib MET
Amuvatinib PDGFRB
Amuvatinib PDGFRA
Midostaurin FGFR1
Midostaurin PDGFRB
Midostaurin PDGFRA
MK-2206 EGFR
Ponatinib EGFR
Ponatinib FGFR1
Ponatinib MET
Ponatinib FGFR4
Ponatinib FGFR2
Ponatinib PDGFRB
Ponatinib PDGFRA
Ola

## Visualization

In [30]:
def draw_selected_drugs(rank_dict, sal_dict, gene_list, top_n=100):
    r, c = len(sal_dict), 1
    fig = plt.figure(1, figsize=[len(sal_dict)*15, top_n])

    i = 0
    for key in sal_dict.keys():
        i += 1
        rnk = rank_dict[key]
        sal_score = sal_dict[key].reshape(-1)
        ranked_ss = sal_score[rnk]
        ranked_genes = gene_list[rnk]
        
        values = ranked_ss[:top_n]
        index = ranked_genes[:top_n]
        
        plt.subplot(r, c, i)
#         sns.set(font_scale=10)
#         plt.title(key, fontdict={'fontsize': 30})
        res = sns.heatmap(values.reshape(1, -1), annot=True, annot_kws={'fontsize':50, 'rotation':"vertical",'verticalalignment':"center"}, cmap="YlGnBu")
        res.set_xticklabels(index, fontdict={'fontsize':50}, rotation=45)
        res.set_yticklabels([key], fontdict={'fontsize':50})
        
    plt.show()
    
    return fig

In [45]:
# fig = draw_selected_drugs(selected_rank_dict, selected_sal_dict, gene_list, 100)

In [32]:
save_path = path + '/SaliencyMap/IG/CellLine/' + model_name
fig.savefig(save_path + '/sensitive_drugs_gene_saliency.png')